# 1. Importar as bibliotecas necessárias

In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import io
import numpy as np

# 2. Baixar os pacotes do NLTK (para stopwords e tokenização)

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

# 3. Ler o dataset (arquivo .csv)

In [5]:
futebol = 'Jogadores e Características .csv'

df = pd.read_csv(futebol)
print(df)

           Jogador                                          Descricao
0    Kylian Mbappé  Rápido, bom drible, joga pela esquerda, finali...
1      Vinicius Jr  Muito rápido, drible ousado, joga pela esquerd...
2  Marcus Rashford  Atacante rápido e versátil, pode jogar pela es...
3   Erling Haaland  Atacante centralizado, finalizador letal. Muit...
4  Kevin De Bruyne  Meio-campista criativo, passe excelente. Visão...
5     N'Golo Kanté  Meio-campista defensivo, incansável. Excelente...
6     Lionel Messi  Drible curto e controle de bola incríveis. Vis...
7      Bukayo Saka  Jovem, rápido, joga pela direita. Bom drible, ...


# 4. Limpar o Dataset (NLTK e Re)

Remover pontuação (re).

Converter tudo para minúsculas.

Remover stopwords em português (nltk).

In [8]:
stop_words = set(stopwords.words('portuguese'))

def limpar_texto(texto):
    texto = re.sub(r'[^a-zA-Z\s]', '', texto, re.I|re.A)
    texto = texto.lower()
    palavras = word_tokenize(texto, language='portuguese')
    palavras_limpas = [palavra for palavra in palavras if palavra not in stop_words]
    return ' '.join(palavras_limpas)


df['Descricao_Limpa'] = df['Descricao'].apply(limpar_texto)
print("\n--- Dataset com Descrição Limpa ---")
print(df[['Jogador', 'Descricao_Limpa']])


--- Dataset com Descrição Limpa ---
           Jogador                                    Descricao_Limpa
0    Kylian Mbappé  rpido bom drible joga esquerda finalizador let...
1      Vinicius Jr  rpido drible ousado joga esquerda forte contra...
2  Marcus Rashford  atacante rpido verstil pode jogar esquerda cen...
3   Erling Haaland  atacante centralizado finalizador letal muita ...
4  Kevin De Bruyne  meiocampista criativo passe excelente viso jog...
5     N'Golo Kanté  meiocampista defensivo incansvel excelente des...
6     Lionel Messi  drible curto controle bola incrveis viso jogo ...
7      Bukayo Saka  jovem rpido joga direita bom drible trabalhado...


# 5. Bag of Words (TfidfVectorizer)

##Transformar as descrições limpas em vetores numéricos.

##O TfidfVectorizer criará uma matriz onde:

As linhas são os jogadores.

As colunas são as palavras únicas do vocabulário (ex: "rápido", "drible", "finalizador").

O valor em cada célula é o "peso" (importância) daquela palavra para aquele jogador.

In [9]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Descricao_Limpa'])

print("\n--- Matriz TF-IDF (Shape) ---")
print(f"Shape da Matriz: {tfidf_matrix.shape}")


--- Matriz TF-IDF (Shape) ---
Shape da Matriz: (8, 59)


#6. Similaridade de Cosseno (Sklearn)
##Usando a Similaridade de Cosseno para calcular o cosseno do ângulo entre os vetores de dois jogadores:

Similaridade 1.0 (Ângulo 0°): Vetores idênticos.

Similaridade 0.0 (Ângulo 90°): Vetores totalmente diferentes (ortogonais).

In [10]:
cosine_sim_matrix = cosine_similarity(tfidf_matrix)

print("\n--- Matriz de Similaridade de Cosseno (8x8) ---")
print("Cada linha/coluna corresponde a um jogador na ordem do dataset.")
print(cosine_sim_matrix)


--- Matriz de Similaridade de Cosseno (8x8) ---
Cada linha/coluna corresponde a um jogador na ordem do dataset.
[[1.         0.21422687 0.1876659  0.28235209 0.09677003 0.05316073
  0.15369045 0.24273075]
 [0.21422687 1.         0.09790707 0.07212357 0.03602093 0.07868824
  0.07890291 0.14016382]
 [0.1876659  0.09790707 1.         0.20922503 0.29272797 0.
  0.05424194 0.20391054]
 [0.28235209 0.07212357 0.20922503 1.         0.09826319 0.04184443
  0.14302482 0.        ]
 [0.09677003 0.03602093 0.29272797 0.09826319 1.         0.12799419
  0.19567555 0.04081369]
 [0.05316073 0.07868824 0.         0.04184443 0.12799419 1.
  0.         0.        ]
 [0.15369045 0.07890291 0.05424194 0.14302482 0.19567555 0.
  1.         0.16749106]
 [0.24273075 0.14016382 0.20391054 0.         0.04081369 0.
  0.16749106 1.        ]]


# 7. Matriz de Similaridade para Interação

##Este código abaixo vai:

Listar os jogadores disponíveis.

Pedir para o usuário digitar um nome.

Encontrar os jogadores mais similares.

Mostrar a Similaridade (de 0.0 a 1.0) e o Ângulo (de 90° a 0°), assim como no seu vídeo de exemplo!

In [11]:
indices = pd.Series(df.index, index=df['Jogador']).drop_duplicates()

def get_recomendacoes(nome_jogador, cosine_sim_matrix, df_indices):
    try:
        idx = df_indices[nome_jogador]
    except KeyError:
        print(f"Erro: Jogador '{nome_jogador}' não encontrado no dataset.")
        return

    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_similares = sim_scores[1:4]

    print(f"\n⚡ Recomendações baseadas em '{nome_jogador}':\n")

    for i, score in top_similares:
        angle_rad = np.arccos(np.clip(score, -1.0, 1.0))
        angle_deg = np.degrees(angle_rad)

        if score > 0.35:
            nivel = "(Alta)"
        elif score > 0.1:
            nivel = "(Média)"
        else:
            nivel = "(Baixa)"

        print(f"  {df['Jogador'].iloc[i]:<15} -> Similaridade: {score:.2f} {nivel} \t| Ângulo: {angle_deg:.1f}°")


print("--- ⚽ Comparador de Jogadores por Características ⚽ ---")
print("\nJogadores disponíveis:")

for jogador in df['Jogador']:
    print(f"- {jogador}")

print("\nDigite o nome de um jogador para ver recomendações.")
print("Digite 'sair' para encerrar o programa.")

while True:
    jogador_input = input("\nEscolha um jogador: ")

    if jogador_input.lower() == 'sair':
        print("Encerrando o programa. Até mais!")
        break

    jogador_input = jogador_input.title()

    if jogador_input in indices:
        get_recomendacoes(jogador_input, cosine_sim_matrix, indices)
    else:
        print(f"Jogador '{jogador_input}' não encontrado. Verifique a ortografia e tente novamente.")

--- ⚽ Comparador de Jogadores por Características ⚽ ---

Jogadores disponíveis:
- Kylian Mbappé
- Vinicius Jr
- Marcus Rashford
- Erling Haaland
- Kevin De Bruyne
- N'Golo Kanté
- Lionel Messi
- Bukayo Saka

Digite o nome de um jogador para ver recomendações.
Digite 'sair' para encerrar o programa.

Escolha um jogador: Bukayo Saka

⚡ Recomendações baseadas em 'Bukayo Saka':

  Kylian Mbappé   -> Similaridade: 0.24 (Média) 	| Ângulo: 76.0°
  Marcus Rashford -> Similaridade: 0.20 (Média) 	| Ângulo: 78.2°
  Lionel Messi    -> Similaridade: 0.17 (Média) 	| Ângulo: 80.4°

Escolha um jogador: sair
Encerrando o programa. Até mais!
